# Train your first 🐸 TTS model 💫

### 👋 Hello and welcome to Coqui (🐸) TTS

The goal of this notebook is to show you a **typical workflow** for **training** and **testing** a TTS model with 🐸.

Let's train a very small model on a very small amount of data so we can iterate quickly.

In this notebook, we will:

1. Download data and format it for 🐸 TTS.
2. Configure the training and testing runs.
3. Train a new model.
4. Test the model and display its performance.

So, let's jump right in!


In [ ]:
## Install Coqui TTS
! pip install -U pip
! pip install TTS

## ✅ Data Preparation

### **First things first**: we need some data.

We're training a Text-to-Speech model, so we need some _text_ and we need some _speech_. Specificially, we want _transcribed speech_. The speech must be divided into audio clips and each clip needs transcription. More details about data requirements such as recording characteristics, background noise and vocabulary coverage can be found in the [🐸TTS documentation](https://tts.readthedocs.io/en/latest/formatting_your_dataset.html).

If you have a single audio file and you need to **split** it into clips. It is also important to use a lossless audio file format to prevent compression artifacts. We recommend using **wav** file format.

The data format we will be adopting for this tutorial is taken from the widely-used  **LJSpeech** dataset, where **waves** are collected under a folder:

<span style="color:purple;font-size:15px">
/wavs<br /> 
 &emsp;| - audio1.wav<br /> 
 &emsp;| - audio2.wav<br /> 
 &emsp;| - audio3.wav<br /> 
  ...<br /> 
</span>

and a **metadata.csv** file will have the audio file name in parallel to the transcript, delimited by `|`: 
 
<span style="color:purple;font-size:15px">
# metadata.csv <br /> 
audio1|This is my sentence. <br /> 
audio2|This is maybe my sentence. <br /> 
audio3|This is certainly my sentence. <br /> 
audio4|Let this be your sentence. <br /> 
...
</span>

In the end, we should have the following **folder structure**:

<span style="color:purple;font-size:15px">
/MyTTSDataset <br /> 
&emsp;| <br /> 
&emsp;| -> metadata.csv<br /> 
&emsp;| -> /wavs<br /> 
&emsp;&emsp;| -> audio1.wav<br /> 
&emsp;&emsp;| -> audio2.wav<br /> 
&emsp;&emsp;| ...<br /> 
</span>

🐸TTS already provides tooling for the _LJSpeech_. if you use the same format, you can start training your models right away. <br /> 

After you collect and format your dataset, you need to check two things. Whether you need a **_formatter_** and a **_text_cleaner_**. <br /> The **_formatter_** loads the text file (created above) as a list and the **_text_cleaner_** performs a sequence of text normalization operations that converts the raw text into the spoken representation (e.g. converting numbers to text, acronyms, and symbols to the spoken format).

If you use a different dataset format then the LJSpeech or the other public datasets that 🐸TTS supports, then you need to write your own **_formatter_** and  **_text_cleaner_**.

## ⏳️ Loading your dataset
Load one of the dataset supported by 🐸TTS.

We will start by defining dataset config and setting LJSpeech as our target dataset and define its path.


In [2]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)
    

In [ ]:
# import urllib.request
# import tarfile

# # Download the LJSpeech dataset
# url = "https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2"
# dataset_path = os.path.join(output_path, "LJSpeech-1.1.tar.bz2")
# urllib.request.urlretrieve(url, dataset_path)
# print("Download complete.")

Download complete.


In [ ]:
# # Extract the dataset
# with tarfile.open(dataset_path, "r:bz2") as tar:
#     tar.extractall(path=output_path)
# print("Extraction complete.")

Extraction complete.


In [3]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "LJSpeech-1.1/")
)
print(dataset_config)

BaseDatasetConfig(formatter='ljspeech', dataset_name='', path='tts_train_dir\\LJSpeech-1.1/', meta_file_train='metadata.csv', ignored_speakers=None, language='', phonemizer='', meta_file_val='', meta_file_attn_mask='')


## ✅ Train a new model

Let's kick off a training run 🚀🚀🚀.

Deciding on the model architecture you'd want to use is based on your needs and available resources. Each model architecture has it's pros and cons that define the run-time efficiency and the voice quality.
We have many recipes under `TTS/recipes/` that provide a good starting point. For this tutorial, we will be using `GlowTTS`.

We will begin by initializing the model training configuration.

In [4]:
# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)

Next we will initialize the audio processor which is used for feature extraction and audio I/O.

In [5]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)
# Modify sample rate if for a custom audio dataset:
# ap.sample_rate = 22050


 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


Next we will initialize the tokenizer which is used to convert text to sequences of token IDs.  If characters are not defined in the config, default characters are passed to the config.

In [6]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

Next we will load data samples. Each sample is a list of ```[text, audio_file_path, speaker_name]```. You can define your custom sample loader returning the list of samples.

In [7]:
from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 2908 files in C:\Users\jerem\OneDrive\Documents\Github Projects\Kurisu\TTS\notebooks\tts_train_dir\LJSpeech-1.1


Now we're ready to initialize the model.

Models take a config object and a speaker manager as input. Config defines the details of the model like the number of layers, the size of the embedding, etc. Speaker manager is used by multi-speaker models.

In [8]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training, distributed training, etc.

In [9]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Num. of CPUs: 24
 | > Num. of Torch Threads: 12
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=tts_train_dir\run-November-10-2024_12+28PM-dbf1a08a

 > Model has 28610257 parameters


### AND... 3,2,1... START TRAINING 🚀🚀🚀

In [10]:
trainer.fit()


 > EPOCH: 0/100
 --> tts_train_dir\run-November-10-2024_12+28PM-dbf1a08a

 > TRAINING (2024-11-10 12:29:12) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
| > Number of instances : 2879
 | > Preprocessing samples
 | > Max text length: 186
 | > Min text length: 14
 | > Avg text length: 102.43383119138589
 | 
 | > Max audio length: 222643.0
 | > Min audio length: 24755.0
 | > Avg audio length: 146371.36123654046
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



   --> TIME: 2024-11-10 12:29:28 -- STEP: 0/90 -- GLOBAL_STEP: 0
     | > current_lr: 2.5e-07 
     | > step_time: 0.4978  (0.49783945083618164)
     | > loader_time: 15.805  (15.80497145652771)

 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
c:\Users\jerem\anaconda3\envs\kurisuenv\lib\site-packages\TTS\tts\models\glow_tts.py:415: FutureWarning: `torch.cuda.amp.au



> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
| > Number of instances : 29
 | > Preprocessing samples
 | > Max text length: 176
 | > Min text length: 30
 | > Avg text length: 106.20689655172414
 | 
 | > Max audio length: 220083.0
 | > Min audio length: 39859.0
 | > Avg audio length: 151739.8275862069
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0009982585906982422 (+0)
     | > avg_loss: 3.9584062099456787 (+0)
     | > avg_log_mle: 0.8315806984901428 (+0)
     | > avg_loss_dur: 3.1268255710601807 (+0)

 > BEST MODEL : tts_train_dir\run-November-10-2024_12+28PM-dbf1a08a\best_model_90.pth

 > EPOCH: 1/100
 --> tts_train_dir\run-November-10-2024_12+28PM-dbf1a08a

 > TRAINING (2024-11-10 12:34:34) 
c:\Users\jerem\anaconda3\envs\kurisuenv\lib\site-packages\TTS\tts\models\glow_tts.py:415: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=False):  # avoid mixed_precision in criterion

   --> TIME: 2024-11-10 12:35:06 -- STEP: 10/90 -- GLOBAL_STEP: 100
     | > loss: 3.8814797401428223  (3.912092113494873)
     | > log_mle: 0.8352751135826111  (0.8272572457790375)
     | > loss_dur: 3.0462045669555664  (3.084834837913513)
     | > grad_norm: tensor(10.1297)  (tensor(10.1570))
     | > c

...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0009937286376953125 (-5.0067901611328125e-06)
     | > avg_loss: 0.4452313184738159 (-0.01285552978515625)
     | > avg_log_mle: -0.20860207080841064 (-0.003352642059326172)
     | > avg_loss_dur: 0.6538333892822266 (-0.009502887725830078)


 > EPOCH: 98/100
 --> tts_train_dir\run-November-10-2024_12+28PM-dbf1a08a

 > TRAINING (2024-11-10 19:29:00) 

   --> TIME: 2024-11-10 19:29:16 -- STEP: 5/90 -- GLOBAL_STEP: 8825
     | > loss: 0.438260018825531  (0.43646626472473143)
     | > log_mle: -0.1818838119506836  (-0.17893426418304442)
     | > loss_dur: 0.6201438307762146  (0.6154005289077759)
     | > grad_norm: tensor(4.5320)  (tensor(3.3890))
     | > current_lr: 2.45e-05 
     | > step_time: 0.8534  (0.8063931465148926)
     | > loader_time: 0.001  (0.18772830963134765)


   --> TIME: 2024-11-10 19:29:49 -- STEP: 30/90 -- GLOBAL_STEP: 8850
     | > loss: 0.4909372329711914  (0.4597393810749054)
     | > log_mle: -0.1889437437057495 

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0009996891021728516 (+5.9604644775390625e-06)
     | > avg_loss: 0.4640078544616699 (+0.018776535987854004)
     | > avg_log_mle: -0.19805073738098145 (+0.0105513334274292)
     | > avg_loss_dur: 0.6620585918426514 (+0.008225202560424805)


 > EPOCH: 99/100
 --> tts_train_dir\run-November-10-2024_12+28PM-dbf1a08a

 > TRAINING (2024-11-10 19:32:26) 

   --> TIME: 2024-11-10 19:32:53 -- STEP: 15/90 -- GLOBAL_STEP: 8925
     | > loss: 0.45179545879364014  (0.44792364835739135)
     | > log_mle: -0.1980346441268921  (-0.18327810764312744)
     | > loss_dur: 0.6498301029205322  (0.6312017560005188)
     | > grad_norm: tensor(2.9458)  (tensor(2.9142))
     | > current_lr: 2.475e-05 
     | > step_time: 1.2291  (0.9881466229756674)
     | > loader_time: 0.0022  (0.06326206525166829)


   --> TIME: 2024-11-10 19:33:32 -- STEP: 40/90 -- GLOBAL_STEP: 8950
     | > loss: 0.4756312370300293  (0.4567133724689484)
     | > log_mle: -0.1914736032485

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0009992122650146484 (-4.76837158203125e-07)
     | > avg_loss: 0.4581262469291687 (-0.005881607532501221)
     | > avg_log_mle: -0.19324946403503418 (+0.004801273345947266)
     | > avg_loss_dur: 0.6513757109642029 (-0.010682880878448486)



In [11]:
print(trainer)

#### 🚀 Run the Tensorboard. 🚀
On the notebook and Tensorboard, you can monitor the progress of your model. Also Tensorboard provides certain figures and sample outputs.

In [ ]:
# !pip install tensorboard
# !tensorboard --logdir=tts_train_dir

## ✅ Test the model

We made it! 🙌

Let's kick off the testing run, which displays performance metrics.

We're committing the cardinal sin of ML 😈 (aka - testing on our training data) so you don't want to deploy this model into production. In this notebook we're focusing on the workflow itself, so it's forgivable 😇

You can see from the test output that our tiny model has overfit to the data, and basically memorized this one sentence.

When you start training your own models, make sure your testing data doesn't include your training data 😅

Let's get the latest saved checkpoint. 

In [12]:
import glob, os
output_path = "tts_train_dir"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])

In [19]:
ckpts

['tts_train_dir\\run-November-10-2024_12+28PM-dbf1a08a\\best_model.pth',
 'tts_train_dir\\run-November-10-2024_12+28PM-dbf1a08a\\best_model_8640.pth',
 'tts_train_dir\\run-November-10-2024_12+28PM-dbf1a08a\\checkpoint_4000.pth',
 'tts_train_dir\\run-November-10-2024_12+28PM-dbf1a08a\\checkpoint_5000.pth',
 'tts_train_dir\\run-November-10-2024_12+28PM-dbf1a08a\\checkpoint_6000.pth',
 'tts_train_dir\\run-November-10-2024_12+28PM-dbf1a08a\\checkpoint_7000.pth',
 'tts_train_dir\\run-November-10-2024_12+28PM-dbf1a08a\\checkpoint_8000.pth']

In [20]:
print(configs)

['tts_train_dir\\run-November-10-2024_10+18AM-dbf1a08a\\config.json', 'tts_train_dir\\run-November-10-2024_12+28PM-dbf1a08a\\config.json']


In [34]:
import subprocess

test_ckpt = "tts_train_dir\\run-November-10-2024_12+28PM-dbf1a08a\\best_model.pth"
test_config = "tts_train_dir\\run-November-10-2024_12+28PM-dbf1a08a\\config.json"

subprocess.run([
    "tts",
    "--text", "Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition|Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition",
    "--model_path", test_ckpt,
    "--config_path", test_config,
    "--out_path", "out.wav"
])

CompletedProcess(args=['tts', '--text', 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition|Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition', '--model_path', 'tts_train_dir\\run-November-10-2024_12+28PM-dbf1a08a\\best_model.pth', '--config_path', 'tts_train_dir\\run-November-10-2024_12+28PM-dbf1a08a\\config.json', '--out_path', 'out.wav'], returncode=0)

In [ ]:
 !tts --text "Text for TTS" \
      --model_path $test_ckpt \
      --config_path $test_config \
      --out_path out.wav

Traceback (most recent call last):
  File "C:\Users\jerem\anaconda3\envs\kurisuenv\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\jerem\anaconda3\envs\kurisuenv\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\jerem\anaconda3\envs\kurisuenv\Scripts\tts.exe\__main__.py", line 7, in <module>
  File "C:\Users\jerem\anaconda3\envs\kurisuenv\lib\site-packages\TTS\bin\synthesize.py", line 423, in main
    synthesizer = Synthesizer(
  File "C:\Users\jerem\anaconda3\envs\kurisuenv\lib\site-packages\TTS\utils\synthesizer.py", line 93, in __init__
    self._load_tts(tts_checkpoint, tts_config_path, use_cuda)
  File "C:\Users\jerem\anaconda3\envs\kurisuenv\lib\site-packages\TTS\utils\synthesizer.py", line 183, in _load_tts
    self.tts_config = load_config(tts_config_path)
  File "C:\Users\jerem\anaconda3\envs\kurisuenv\lib\site-packages\TTS\config\__init__.py", line 94, in load_config
    raise Type

## 📣 Listen to the synthesized wave 📣

In [35]:
import IPython
IPython.display.Audio("out.wav")

## 🎉 Congratulations! 🎉 You now have trained your first TTS model! 
Follow up with the next tutorials to learn more advanced material.